<a href="https://colab.research.google.com/github/takiguchi-yu/CodingLab/blob/main/RAG_over_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain GitPython

# Set env var OPENAI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_dotenv()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from git import Repo
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
import os

# Git Clone
repo_path = "./repo"
if not os.path.exists(repo_path):
  Repo.clone_from("https://github.com/takiguchi-yu/cognito-google-federation-example", to_path=repo_path)
else:
  repo = Repo(repo_path)
  repo.remotes[0].pull()



In [ ]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py", ".js", ".ts", ".tsx", ".css", ".php", ".md", ".json", ".html"],
    exclude=["**/.git"],
    parser=LanguageParser()
)
docs = loader.load()
len(docs)

39

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=200, chunk_size=2048)
texts = text_splitter.split_documents(docs)
len(texts)

170

In [ ]:
!wget https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf

--2024-02-18 15:06:43--  https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 13.224.50.5, 13.224.50.55, 13.224.50.80, ...
Connecting to huggingface.co (huggingface.co)|13.224.50.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7b/80/7b800b9247db84b65d4ca008ca4433a306b7f259163c4d026874b4aa9f7112eb/48cc5600c5e35b1226208a53b1871f50efb15764232babaef23e2264c285d7d9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27codellama-13b-instruct.Q4_K_M.gguf%3B+filename%3D%22codellama-13b-instruct.Q4_K_M.gguf%22%3B&Expires=1708527630&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODUyNzYzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Yi84MC83YjgwMGI5MjQ3ZGI4NGI2NWQ0Y2EwMDhjYTQ0MzNhMzA2YjdmMjU5MTYzYzRkMDI2ODc0YjRhYTlmNzExMmViLzQ4Y2M1NjAwYzVlMzViMTIyNj

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 180.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 293.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 237.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 268.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.44-cp310-cp310-manylinux_2_35_x86_64.whl size=20499591 sha256=22792e2d16a300972e86a60504596517d633bb386375b6cab75e4c10db29e5cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-lw97s7b0/wheels/6e/f0/52/1716aa7fefc7eb2a9b76775b0a61fc131b7dcc961e310a048a
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4

In [ ]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 11.9 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import GPT4AllEmbeddings, LlamaCppEmbeddings, GPT4AllEmbeddings

db = Chroma.from_documents(texts, embedding = GPT4AllEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

100%|██████████| 45.9M/45.9M [00:01<00:00, 43.5MiB/s]


In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="codellama-13b-instruct.Q4_K_M.gguf",
    n_ctx=5000,
    n_gpu_layers=30,
    n_batch=512,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from codellama-13b-instruct.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = codellama_codellama-13b-instruct-hf
llama_model_loader: - kv   2:                       llama.context_length u32              = 16384
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atten

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "What is BackendAPI?"
result = qa(question)
result['answer']

Llama.generate: prefix-match hit


 What is BackendAPI?

I have this question: What is BackendAPI?
I have this question: What is BackendAPI?
Comment: What is BackendAPI?


llama_print_timings:        load time =    1768.56 ms
llama_print_timings:      sample time =      19.89 ms /    41 runs   (    0.49 ms per token,  2061.13 tokens per second)
llama_print_timings: prompt eval time =    2615.58 ms /    10 tokens (  261.56 ms per token,     3.82 tokens per second)
llama_print_timings:        eval time =   13932.40 ms /    40 runs   (  348.31 ms per token,     2.87 tokens per second)
llama_print_timings:       total time =   16821.06 ms /    50 tokens
Llama.generate: prefix-match hit





Back is:

Back:


Back:
This


Back:



    Back:


Back is: Back
        is:
    Back

   

   




   
   
       
   
   

   
   
   :
   
   :

   
   :
    
   

   :
   :
   
   :

   :

   









   .




   


   







or
   






   
   


   
   





   
   
   
    
       
                

   














llama_print_timings:        load time =    1768.56 ms
llama_print_timings:      sample time =      98.27 ms /   186 runs   (    0.53 ms per token,  1892.84 tokens per second)
llama_print_timings: prompt eval time =   55581.39 ms /  4813 tokens (   11.55 ms per token,    86.59 tokens per second)
llama_print_timings:        eval time =   90812.83 ms /   185 runs   (  490.88 ms per token,     2.04 tokens per second)
llama_print_timings:       total time =  147789.39 ms /  4998 tokens
Llama.generate: prefix-match hit



The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.


llama_print_timings:        load time =    1768.56 ms
llama_print_timings:      sample time =      17.44 ms /    35 runs   (    0.50 ms per token,  2007.34 tokens per second)
llama_print_timings: prompt eval time =    4057.04 ms /   610 tokens (    6.65 ms per token,   150.36 tokens per second)
llama_print_timings:        eval time =   11949.55 ms /    34 runs   (  351.46 ms per token,     2.85 tokens per second)
llama_print_timings:       total time =   16221.39 ms /   644 tokens


'\n\n\nBack is:\n\nBack:\n\n\nBack:\nThis\n\n\nBack:\n\n\n\n    Back:\n\n\nBack is: Back\n        is:\n    Back\n\n   \n\n   \n\n\n\n\n   \n   \n       \n   \n   \n\n   \n   \n   :\n   \n   :\n\n   \n   :\n    \n   \n\n   :\n   :\n   \n   :\n\n   :\n\n   \n\n\n\n\n\n\n\n\n\n   .\n\n\n\n\n   \n\n\n   \n\n\n\n\n\n\n\nor\n   \n\n\n\n\n\n\n   \n   \n\n\n   \n   \n\n\n\n\n\n   \n   \n   \n    \n       \n                \n\n   \n\n\n\n\n\n\n\n\n\n\n\n'